In [18]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(),os.pardir))
import altair as alt
import pandas as pd
import warnings
warnings.simplefilter("ignore", UserWarning)
data_path = os.path.join(os.pardir, "data", "processed", "preprocessed-complaints.csv")

complaints_df = pd.read_csv(data_path, index_col=False)

/var/folders/qz/pdbq5w0j37b7r1w67lr3lk240000gn/T/ipykernel_53327/3454067259.py:10: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  complaints_df = pd.read_csv(data_path, index_col=False)


In [19]:

# complaints_df = complaints_df.iloc[: , 1:]
complaints_df.head()


,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id
0,2022-11-22,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,The unverified accounts indicated below contin...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,FL,32839.0,NaN,Consent provided,Web,2022-11-22,Closed with non-monetary relief,Yes,NaN,6227407
1,2022-11-29,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Investigation took more than 30 days,NaN,NaN,"EQUIFAX, INC.",CA,90045.0,NaN,NaN,Web,2022-11-29,In progress,Yes,NaN,6254497
2,2022-11-29,Debt collection,Medical debt,Attempts to collect debt not owed,Debt is not yours,NaN,Company believes it acted appropriately as aut...,"Valley Collection Service, LLC",AZ,85234.0,NaN,Other,Web,2022-11-29,Closed with explanation,Yes,NaN,6250329
3,2022-11-29,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,"EQUIFAX, INC.",GA,30097.0,NaN,NaN,Web,2022-11-29,In progress,Yes,NaN,6254320
4,2022-11-28,Debt collection,Credit card debt,Communication tactics,Frequent or repeated calls,NaN,Company has responded to the consumer and the ...,"CITIBANK, N.A.",GA,30315.0,NaN,Consent not provided,Web,2022-11-28,Closed with explanation,Yes,NaN,6251994


In [20]:
complaints_df = complaints_df.query('not consumer_disputed.isnull()')
complaints_df['consumer_disputed'].replace(['Yes','No'],[1,0], inplace = True)
drop_features = ['date_received',
                 'zip_code',
                 'tags',
                 'date_sent_to_company',
                 'complaint_id']
complaints_df = complaints_df.drop(columns = drop_features).dropna()

In [21]:
complaints_df.head()
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(complaints_df,test_size=0.2, random_state=123)

In [22]:
train_df.head()

,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,consumer_consent_provided,submitted_via,company_response_to_consumer,timely_response,consumer_disputed
2278113,Debt collection,I do not know,False statements or representation,Indicated committed crime not paying,navy federal credit union has continually to v...,Company believes it acted appropriately as aut...,NAVY FEDERAL CREDIT UNION,NC,Consent provided,Web,Closed with explanation,Yes,1
1155179,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt was paid,I have submitted a separate complaint on XXXX ...,Company believes it acted appropriately as aut...,"Stellar Recovery, Inc.",IN,Consent provided,Web,Closed with explanation,Yes,1
2144705,Debt collection,Medical,Communication tactics,Used obscene/profane/abusive language,Representative by the name if XXXX XXXX called...,Company believes it acted appropriately as aut...,"Capital Accounts, LLC",LA,Consent provided,Web,Closed with explanation,Yes,0
2003993,Debt collection,Medical,Cont'd attempts collect debt not owed,Debt was paid,I contacted the debtor to let them know that a...,Company has responded to the consumer and the ...,TENET HEALTHCARE CORPORATION,AL,Consent provided,Web,Closed with explanation,Yes,0
2271766,Debt collection,I do not know,Cont'd attempts collect debt not owed,Debt is not mine,This company continues to report on my credit ...,Company believes it acted appropriately as aut...,"Amsher Collection Services, Inc.",TX,Consent provided,Web,Closed with explanation,Yes,0


In [23]:
unique_df = pd.DataFrame()
unique_df['columns'] = complaints_df.columns
unique_df['valid_count'] = complaints_df.count(axis=0).reset_index()[0]
unique_df['unique_count'] = complaints_df.nunique().reset_index()[0]
unique_df

,columns,valid_count,unique_count
0,product,19432,2
1,sub_product,19432,10
2,issue,19432,9
3,sub_issue,19432,37
4,consumer_complaint_narrative,19432,19146
5,company_public_response,19432,10
6,company,19432,1425
7,state,19432,57
8,consumer_consent_provided,19432,1
9,submitted_via,19432,1


In [24]:
target = pd.DataFrame(complaints_df.value_counts('consumer_disputed')).reset_index()
target.columns = ['consumer_disputed','count']
alt.Chart(target).mark_bar().encode(
    x=alt.X('consumer_disputed:O',title = 'Consumer Disputed'),
    y=alt.Y('count:Q',title = 'Count'),
    color='consumer_disputed:O',
)

alt.Chart(...)

In [25]:
X_train, y_train = train_df.drop(columns= ['consumer_disputed']), train_df['consumer_disputed']
X_test, y_test = test_df.drop(columns= ['consumer_disputed']), test_df['consumer_disputed']

In [26]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate

categorical_features = ['product',
                        'sub_product',
                        'issue',
                        'sub_issue',
                        'company_public_response', 
                        'company',
                        'state',
                        'consumer_consent_provided',
                        'consumer_consent_provided',
                        'submitted_via',
                        'company_response_to_consumer',
                        'timely_response']
drop_features = ['consumer_consent_provided','submitted_via']


text_feature = 'consumer_complaint_narrative'

preprocessor = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'ignore',
                   drop='if_binary'), categorical_features),
    (CountVectorizer(stop_words='english', max_features = 1000), text_feature),
    ('drop', drop_features))
scoring_metrics = ['accuracy','recall','precision','f1']

cross_val_results = {}


In [27]:

from sklearn.dummy import DummyClassifier
pipe_dc = make_pipeline(preprocessor, DummyClassifier())
pipe_dc.fit(X_train, y_train)
cross_val_results['dummy'] = pd.DataFrame(cross_validate(
    pipe_dc, X_train, y_train,scoring=scoring_metrics)).agg(['mean']).round(3).T

In [28]:
cross_val_results['dummy']

,mean
fit_time,2.383
score_time,0.570
test_accuracy,0.778
test_recall,0.000
test_precision,0.000
test_f1,0.000


In [29]:
from sklearn.linear_model import LogisticRegression
pipe_lr = make_pipeline(preprocessor, LogisticRegression(max_iter=1000, class_weight='balanced'))
cross_val_results['logreg'] = pd.DataFrame(cross_validate(
    pipe_lr, X_train, y_train, n_jobs=-1, scoring=scoring_metrics)).agg(['mean']).round(3).T
cross_val_results['logreg']

/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

,mean
fit_time,9.912
score_time,1.078
test_accuracy,0.631
test_recall,0.506
test_precision,0.303
test_f1,0.379


In [30]:
from sklearn.naive_bayes import BernoulliNB
pipe_nb = make_pipeline(preprocessor, BernoulliNB(alpha = 0.1))
cross_val_results['bayes'] = pd.DataFrame(cross_validate(
    pipe_nb, X_train, y_train, n_jobs=-1, scoring=scoring_metrics)).agg(['mean']).round(3).T
cross_val_results['bayes']

/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/

,mean
fit_time,2.494
score_time,0.472
test_accuracy,0.695
test_recall,0.366
test_precision,0.332
test_f1,0.348


In [31]:
from sklearn.svm import SVC
pipe_svc = make_pipeline(preprocessor, SVC(class_weight='balanced'))
cross_val_results['svc'] = pd.DataFrame(cross_validate(
    pipe_svc, X_train, y_train, n_jobs=-1, scoring=scoring_metrics)).agg(['mean']).round(3).T
cross_val_results['svc']

/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/

,mean
fit_time,88.211
score_time,20.809
test_accuracy,0.676
test_recall,0.467
test_precision,0.335
test_f1,0.391


In [32]:
from sklearn.ensemble import RandomForestClassifier
pipe_rf = make_pipeline(preprocessor, RandomForestClassifier(class_weight='balanced'))
cross_val_results['random forest'] = pd.DataFrame(cross_validate(
    pipe_rf, X_train, y_train, n_jobs=-1, scoring=scoring_metrics)).agg(['mean']).round(3).T
cross_val_results['random forest']

/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/

,mean
fit_time,21.921
score_time,0.632
test_accuracy,0.783
test_recall,0.054
test_precision,0.655
test_f1,0.099


In [33]:
res = pd.concat(cross_val_results, axis=1)

In [34]:
res.columns = res.columns.droplevel(1)

In [35]:
res = res.reset_index()

In [36]:
res[2:]

,index,dummy,logreg,bayes,svc,random forest
2,test_accuracy,0.778,0.631,0.695,0.676,0.783
3,test_recall,0.000,0.506,0.366,0.467,0.054
4,test_precision,0.000,0.303,0.332,0.335,0.655
5,test_f1,0.000,0.379,0.348,0.391,0.099


In [37]:
source = res[2:].melt(id_vars=['index'])

In [38]:
source.columns = ['Metric','Model','Score']
source['Metric'] = source['Metric'].str.replace('test_','')

In [39]:

alt.Chart(source).mark_bar().encode(
    x='Metric:O',
    y='Score:Q',
    color='Metric:N',
    column='Model:N'
).save('test.png')

In [42]:
models = {'Dummy Classifier':pipe_dc, 
          'Logistic Regression':pipe_lr, 
          'Naive Bayes Classifier':pipe_nb,
          'Support Vector Classifier':pipe_svc,
          'Random Forest Classifier':pipe_rf}
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
scores = []
for model in models:
    print(model)
    models[model].fit(X_train, y_train)
    scores.append([accuracy_score(models[model].predict(X_test), y_test),
              recall_score(models[model].predict(X_test), y_test),
              precision_score(models[model].predict(X_test), y_test),
              f1_score(models[model].predict(X_test), y_test)])
    print(scores)
    

Dummy Classifier
[[0.7818368922047851, 0.0, 0.0, 0.0]]
Logistic Regression
[[0.7818368922047851, 0.0, 0.0, 0.0], [0.6449704142011834, 0.311614730878187, 0.5188679245283019, 0.3893805309734513]]
Naive Bayes Classifier
[[0.7818368922047851, 0.0, 0.0, 0.0], [0.6449704142011834, 0.311614730878187, 0.5188679245283019, 0.3893805309734513], [0.7087728325186519, 0.3442982456140351, 0.37028301886792453, 0.3568181818181818]]
Support Vector Classifier
[[0.7818368922047851, 0.0, 0.0, 0.0], [0.6449704142011834, 0.311614730878187, 0.5188679245283019, 0.3893805309734513], [0.7087728325186519, 0.3442982456140351, 0.37028301886792453, 0.3568181818181818], [0.6843323900180087, 0.3333333333333333, 0.44693396226415094, 0.38186397984886644]]
Random Forest Classifier
[[0.7818368922047851, 0.0, 0.0, 0.0], [0.6449704142011834, 0.311614730878187, 0.5188679245283019, 0.3893805309734513], [0.7087728325186519, 0.3442982456140351, 0.37028301886792453, 0.3568181818181818], [0.6843323900180087, 0.3333333333333333, 0

In [43]:
pd.DataFrame(scores, index=models.keys(), columns =['Accuracy','Recall','Precision','F1']).T\
        .to_csv(os.path.join("test_scores.csv"))

In [44]:
pd.DataFrame(scores, index=models.keys(), columns =['Accuracy','Recall','Precision','F1']).T

,Dummy Classifier,Logistic Regression,Naive Bayes Classifier,Support Vector Classifier,Random Forest Classifier
Accuracy,0.781837,0.644970,0.708773,0.684332,0.788783
Recall,0.000000,0.311615,0.344298,0.333333,0.695652
Precision,0.000000,0.518868,0.370283,0.446934,0.056604
F1,0.000000,0.389381,0.356818,0.381864,0.104689
